In [3]:
!pip install sentence-transformers underthesea


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import pickle
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer, util, CrossEncoder
from sentence_transformers.evaluation import BinaryClassificationEvaluator
from sentence_transformers.losses import ContrastiveLoss
from torch.utils.data import DataLoader
from tqdm import tqdm
import sys
from underthesea import word_tokenize
import torch

c:\Users\DELL\anaconda3\envs\dual_retrieval\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%pwd

'e:\\Legal-document-dual-retrieval-approach\\tokenize'

In [3]:
%cd ..

e:\Legal-document-dual-retrieval-approach


c:\Users\DELL\anaconda3\envs\dual_retrieval\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
%pwd

'e:\\Legal-document-dual-retrieval-approach'

In [6]:
corpus = pd.read_csv('data/corpus.csv')
train = pd.read_csv('data/train.csv', on_bad_lines='skip')

In [4]:
nan_qid = train[train['qid'].isna() == True].index
nan_qid

Index([26402], dtype='int64')

In [5]:
train.loc[nan_qid[0], 'qid'] = 0
train['qid'] = train['qid'].astype(int)
train.loc[nan_qid[0], 'qid']

0

In [6]:
train['list_cid'] = train['cid'].apply(lambda x: x.strip('[]').split()).tolist()
train['list_cid'] = train['list_cid'].apply(lambda x: [int(i) for i in x])

In [7]:
documents_dir = dict(zip(corpus['cid'], corpus['text']))
queries_dir = dict(zip(train['qid'], train['question']))
queries_doc = dict(zip(train['qid'], train['list_cid']))

In [8]:
queries_map = dict(zip(train['question'], train['qid']))

In [9]:
documents_id = list(documents_dir.keys())
documents_list = list(documents_dir.values())

In [10]:
queries_id = list(queries_dir.keys())
queries_list = list(queries_dir.values())

In [11]:
def transform_tokenized_to_vncorenlp_form(tokenized_string):
    transformed_list = [token.replace(" ", "_") for token in tokenized_string]
    final_string = " ".join(transformed_list)
    return final_string

In [12]:
tokenized_queries = [word_tokenize(query) for query in queries_list]
transform_tokenized_queries = [transform_tokenized_to_vncorenlp_form(tokenized_query) for tokenized_query in tokenized_queries]

In [13]:
max_tokenized_query_length = max(len(query) for query in transform_tokenized_queries)
print(f"Max tokenized query length: {max_tokenized_query_length}")

Max tokenized query length: 277


In [14]:
tokenized_query_length = [len(query) for query in transform_tokenized_queries]
tokenized_query_length_top10 = sorted(tokenized_query_length, reverse=True)[:10]
print(f"Top 10 tokenized query length: {tokenized_query_length_top10}")

Top 10 tokenized query length: [277, 245, 241, 235, 234, 232, 231, 228, 226, 226]


In [15]:
with open('/kaggle/input/legal-dataset/tokenized_documents.pkl', 'rb') as file:
    tokenized_documents = pickle.load(file)

In [16]:
transform_tokenized_documents = []
for tokenized_doc in tokenized_documents:
    transform_tokenized_documents.append(transform_tokenized_to_vncorenlp_form(tokenized_doc))

In [17]:
max_tokenized_length = max(len(doc.split()) for doc in transform_tokenized_documents)
print(f"Max tokenized length in a sentence: {max_tokenized_length}")

Max tokenized length in a sentence: 52663


In [18]:
tokenized_pairs = [(doc,len(doc.split())) for doc in transform_tokenized_documents]
max_tokenized_pairs = sorted(tokenized_pairs, key=lambda x: x[1], reverse=True)[:1]
max_tokenized_pairs

[('1 . Sửa_đổi , bổ_sung Điều 3 như sau : a ) Sửa_đổi , bổ_sung điểm d khoản 1 Điều 3 như sau : “ d ) Nghiêm_trị người phạm_tội dùng thủ_đoạn xảo_quyệt , có tổ_chức , có tính_chất chuyên_nghiệp , cố_ý gây hậu_quả đặc_biệt nghiêm_trọng . Khoan_hồng đối_với người tự_thú , đầu_thú , thành_khẩn khai_báo , tố_giác đồng_phạm , lập_công chuộc_tội , tích_cực hợp_tác với cơ_quan có trách_nhiệm trong việc phát_hiện tội_phạm hoặc trong quá_trình giải_quyết vụ án , ăn_năn hối_cải , tự_nguyện sửa_chữa hoặc bồi_thường thiệt_hại gây ra ; ” ; b ) Sửa_đổi , bổ_sung điểm d khoản 2 Điều 3 như sau : “ d ) Khoan_hồng đối_với pháp_nhân thương_mại tích_cực hợp_tác với cơ_quan có trách_nhiệm trong việc phát_hiện tội_phạm hoặc trong quá_trình giải_quyết vụ án , tự_nguyện sửa_chữa , bồi_thường thiệt_hại gây ra , chủ_động ngăn_chặn hoặc khắc_phục hậu_quả xảy ra . ” . 2 . Sửa_đổi , bổ_sung Điều 9 như sau : “ Điều 9 . Phân_loại tội_phạm 1 . Căn_cứ vào tính_chất và mức_độ nguy_hiểm cho xã_hội của hành_vi phạm_tội đ

In [19]:
transform_tokenized_documents[0:2]

['Thông_tư này hướng_dẫn tuần_tra , canh_gác bảo_vệ đê_Điều trong mùa lũ đối_với các tuyến đê sông được phân_loại , phân_cấp theo quy_định tại Điều 4 của Luật Đê_Điều .',
 '1 . Hàng năm trước mùa mưa , lũ , Ủy_ban_nhân_dân cấp xã nơi có đê phải tổ_chức lực_lượng lao_động tại địa_phương để tuần_tra , canh_gác đê và thường_trực trên các điếm canh đê hoặc nhà dân khu_vực gần đê ( đối_với những khu_vực chưa có điếm canh đê ) , khi có báo_động lũ từ cấp I trở lên đối_với tuyến sông có đê ( sau đây gọi tắt là lực_lượng tuần_tra , canh_gác đê ) . 2 . Lực_lượng tuần_tra , canh_gác đê được tổ_chức thành các đội , do Ủy_ban_nhân_dân cấp xã ra quyết_định thành_lập ; từ 01 đến 02 kilômét đê thành_lập 01 đội ; mỗi đội có từ 12 đến 18 người , trong đó có 01 đội_trưởng và 01 hoặc 02 đội phó . Danh_sách thành_viên đội tuần_tra , canh_gác đê được niêm_yết tại điếm canh đê thuộc địa_bàn được phân_công . 3 . Khi lũ , bão có diễn_biến phức_tạp , kéo_dài ngày , Ủy_ban_nhân_dân cấp xã có_thể quyết_định việc

In [20]:
from underthesea import sent_tokenize
sentences = [sent_tokenize(doc) for doc in transform_tokenized_documents[0:2]]
sentences

[['Thông_tư này hướng_dẫn tuần_tra , canh_gác bảo_vệ đê_Điều trong mùa lũ đối_với các tuyến đê sông được phân_loại , phân_cấp theo quy_định tại Điều 4 của Luật Đê_Điều .'],
 ['1 .',
  'Hàng năm trước mùa mưa , lũ , Ủy_ban_nhân_dân cấp xã nơi có đê phải tổ_chức lực_lượng lao_động tại địa_phương để tuần_tra , canh_gác đê và thường_trực trên các điếm canh đê hoặc nhà dân khu_vực gần đê ( đối_với những khu_vực chưa có điếm canh đê ) , khi có báo_động lũ từ cấp I trở lên đối_với tuyến sông có đê ( sau đây gọi tắt là lực_lượng tuần_tra , canh_gác đê ) .',
  '2 .',
  'Lực_lượng tuần_tra , canh_gác đê được tổ_chức thành các đội , do Ủy_ban_nhân_dân cấp xã ra quyết_định thành_lập ; từ 01 đến 02 kilômét đê thành_lập 01 đội ; mỗi đội có từ 12 đến 18 người , trong đó có 01 đội_trưởng và 01 hoặc 02 đội phó .',
  'Danh_sách thành_viên đội tuần_tra , canh_gác đê được niêm_yết tại điếm canh đê thuộc địa_bàn được phân_công .',
  '3 .',
  'Khi lũ , bão có diễn_biến phức_tạp , kéo_dài ngày , Ủy_ban_nhân_

In [21]:
import re
def remove_elements(data):
    pattern = re.compile(r'^\d+\s\.$')  # Regex pattern for elements like '2 .' or '3 .'
    return [[item for item in sublist if not pattern.match(item)] for sublist in data]

filtered_data = remove_elements(sentences)
print(filtered_data)

[['Thông_tư này hướng_dẫn tuần_tra , canh_gác bảo_vệ đê_Điều trong mùa lũ đối_với các tuyến đê sông được phân_loại , phân_cấp theo quy_định tại Điều 4 của Luật Đê_Điều .'], ['Hàng năm trước mùa mưa , lũ , Ủy_ban_nhân_dân cấp xã nơi có đê phải tổ_chức lực_lượng lao_động tại địa_phương để tuần_tra , canh_gác đê và thường_trực trên các điếm canh đê hoặc nhà dân khu_vực gần đê ( đối_với những khu_vực chưa có điếm canh đê ) , khi có báo_động lũ từ cấp I trở lên đối_với tuyến sông có đê ( sau đây gọi tắt là lực_lượng tuần_tra , canh_gác đê ) .', 'Lực_lượng tuần_tra , canh_gác đê được tổ_chức thành các đội , do Ủy_ban_nhân_dân cấp xã ra quyết_định thành_lập ; từ 01 đến 02 kilômét đê thành_lập 01 đội ; mỗi đội có từ 12 đến 18 người , trong đó có 01 đội_trưởng và 01 hoặc 02 đội phó .', 'Danh_sách thành_viên đội tuần_tra , canh_gác đê được niêm_yết tại điếm canh đê thuộc địa_bàn được phân_công .', 'Khi lũ , bão có diễn_biến phức_tạp , kéo_dài ngày , Ủy_ban_nhân_dân cấp xã có_thể quyết_định việc 

In [22]:
sentence_tokenized = [sent_tokenize(doc) for doc in transform_tokenized_documents]

In [23]:
sentence_tokenized_filter = remove_elements(sentence_tokenized)

In [24]:
sentence_tokenized_filter[0:2]

[['Thông_tư này hướng_dẫn tuần_tra , canh_gác bảo_vệ đê_Điều trong mùa lũ đối_với các tuyến đê sông được phân_loại , phân_cấp theo quy_định tại Điều 4 của Luật Đê_Điều .'],
 ['Hàng năm trước mùa mưa , lũ , Ủy_ban_nhân_dân cấp xã nơi có đê phải tổ_chức lực_lượng lao_động tại địa_phương để tuần_tra , canh_gác đê và thường_trực trên các điếm canh đê hoặc nhà dân khu_vực gần đê ( đối_với những khu_vực chưa có điếm canh đê ) , khi có báo_động lũ từ cấp I trở lên đối_với tuyến sông có đê ( sau đây gọi tắt là lực_lượng tuần_tra , canh_gác đê ) .',
  'Lực_lượng tuần_tra , canh_gác đê được tổ_chức thành các đội , do Ủy_ban_nhân_dân cấp xã ra quyết_định thành_lập ; từ 01 đến 02 kilômét đê thành_lập 01 đội ; mỗi đội có từ 12 đến 18 người , trong đó có 01 đội_trưởng và 01 hoặc 02 đội phó .',
  'Danh_sách thành_viên đội tuần_tra , canh_gác đê được niêm_yết tại điếm canh đê thuộc địa_bàn được phân_công .',
  'Khi lũ , bão có diễn_biến phức_tạp , kéo_dài ngày , Ủy_ban_nhân_dân cấp xã có_thể quyết_địn

In [25]:
max_sentence_length = max(len(sentence.split()) for document in sentence_tokenized_filter for sentence in document)
print(f'Max sentence length: {max_sentence_length}')

Max sentence length: 1722


In [26]:
sentence_lengths = [len(sentence.split()) for document in sentence_tokenized_filter for sentence in document]

# Find the top 10 maximum lengths
top_10_lengths = sorted(sentence_lengths, reverse=True)[:10]

print("Top 10 maximum lengths:", top_10_lengths)

Top 10 maximum lengths: [1722, 1318, 1171, 1160, 1142, 1141, 1025, 1002, 961, 960]


In [27]:
sentence_length_pairs = [(sentence, len(sentence.split())) for document in sentence_tokenized_filter for sentence in document]
top_10_sentences = sorted(sentence_length_pairs, key=lambda x: x[1], reverse=True)[:10]
print("Top 10 sentences by length:")
for sentence, length in top_10_sentences:
    print(f"Length: {length}, Sentence: {sentence}")

Top 10 sentences by length:
Length: 1722, Sentence: Thẩm_quyền xử_phạt vi_phạm hành_chính của Thanh_tra : a ) Thanh_tra_viên , người được giao thực_hiện nhiệm_vụ thanh_tra chuyên_ngành về hóa_chất đang thi_hành công_vụ xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại khoản 1 và 2 Điều 21 và Điều 36 Nghị_định này ; b ) Chánh Thanh_tra Sở Công_Thương , Trưởng_đoàn thanh_tra chuyên_ngành Sở Công_Thương xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại Điều 5 , 6 , 7 , 8 , 9 , 10 ; khoản 1 , điểm a , b , c , d khoản 2 , khoản 3 , 4 , 5 , 6 và 7 Điều 11 ; Điều 12 , 13 , 14 , 15 , 16 , 17 , 18 ; khoản 1 và điểm a , b khoản 2 Điều 19 ; Điều 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 31 ; khoản 1 và điểm a , b khoản 2 Điều 32 ; Điều 33 ; khoản 1 , 2 , 3 Điều 34 ; Điều 35 , 36 , 49 , 50 ; khoản 1 , 2 và 3 Điều 51 ; khoản 1 , 2 và 3 Điều 52 ; khoản 1 , 2 , 3 , 4 Điều 53 ; khoản 1 , 2 , 3 , 4 , 5 Điều 54 ; khoản 1 , 2 , 3 và 4 Điều 55 ; khoản 1 , 2 , 3 , 4 Điều 56

In [28]:
a = [['Thẩm_quyền xử_phạt vi_phạm hành_chính của Thanh_tra : a ) Thanh_tra_viên , người được giao thực_hiện nhiệm_vụ thanh_tra chuyên_ngành về hóa_chất đang thi_hành công_vụ xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại khoản 1 và 2 Điều 21 và Điều 36 Nghị_định này ; b ) Chánh Thanh_tra Sở Công_Thương , Trưởng_đoàn thanh_tra chuyên_ngành Sở Công_Thương xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại Điều 5 , 6 , 7 , 8 , 9 , 10 ; khoản 1 , điểm a , b , c , d khoản 2 , khoản 3 , 4 , 5 , 6 và 7 Điều 11 ; Điều 12 , 13 , 14 , 15 , 16 , 17 , 18 ; khoản 1 và điểm a , b khoản 2 Điều 19 ; Điều 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 31 ; khoản 1 và điểm a , b khoản 2 Điều 32 ; Điều 33 ; khoản 1 , 2 , 3 Điều 34 ; Điều 35 , 36 , 49 , 50 ; khoản 1 , 2 và 3 Điều 51 ; khoản 1 , 2 và 3 Điều 52 ; khoản 1 , 2 , 3 , 4 Điều 53 ; khoản 1 , 2 , 3 , 4 , 5 Điều 54 ; khoản 1 , 2 , 3 và 4 Điều 55 ; khoản 1 , 2 , 3 , 4 Điều 56 và Điều 57 Nghị_định này ; c ) Chánh Thanh_tra Sở Y_tế , Trưởng_đoàn thanh_tra chuyên_ngành Cục Quản_lý môi_trường y_tế , Trưởng_đoàn thanh_tra chuyên_ngành Sở Y_tế xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại Điều 14 , 15 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 Nghị_định này trong lĩnh_vực y_tế , hóa_chất sử_dụng trong chất diệt khuẩn , diệt côn_trùng trong gia_dụng và y_tế , dược_phẩm và phụ_gia thực_phẩm ; các hành_vi vi_phạm hành_chính quy_định tại Điều 37 ; khoản 1 , 2 , điểm a , b , c , d , đ khoản 3 , điểm a , b , c , d , đ khoản 4 , khoản 5 Điều 38 ; Điều 39 , 40 , 41 , 42 , 43 , 44 , 45 , 46 , 47 và 48 Nghị_định này ; d ) Trưởng_đoàn thanh_tra chuyên_ngành Cục Hóa_chất xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại Điều 5 , 6 , 7 , 8 , 9 , 10 ; khoản 1 , điểm a , b , c , d khoản 2 , khoản 3 , 4 , 5 , 6 và 7 Điều 11 ; Điều 12 , 13 , 14 , 15 , 16 , 17 , 18 ; khoản 1 và điểm a , b khoản 2 Điều 19 ; Điều 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 31 ; khoản 1 và điểm a , b khoản 2 Điều 32 ; Điều 33 ; khoản 1 , 2 , 3 Điều 34 ; Điều 35 , 36 Nghị_định này ; các hành_vi vi_phạm hành_chính về kinh_doanh , xuất_khẩu , nhập_khẩu tiền chất thuốc_nổ , xuất_khẩu , nhập_khẩu vật_liệu nổ công_nghiệp tại điểm b , d , đ khoản 1 , khoản 2 , điểm b , c , d khoản 3 Điều 49 ; Điều 50 ; điểm a_khoản 2 , khoản 3 Điều 51 ; khoản 1 , 2 và 3 Điều 53 và khoản 1 , 2 , 3 , 4 Điều 55 Nghị_định này ; đ ) Trưởng_đoàn thanh_tra chuyên_ngành Cục Kỹ_thuật an_toàn và_Môi_trường công_nghiệp xử_phạt đối_với các hành_vi vi_phạm hành_chính tại Điều 5 ; Điều 8 ; Điều 11 ; Điều 14 , 15 ; khoản 1 Điều 30 ; Điều 49 , 50 ; khoản 1 , 2 và 3 Điều 51 ; khoản 1 , 2 và 3 Điều 52 ; khoản 1 , 2 , 3 , 4 Điều 53 ; khoản 1 , 2 , 3 , 4 , 5 Điều 54 ; khoản 1 , 2 , 3 và 4 Điều 55 ; khoản 1 , 2 , 3 , 4 Điều 56 và Điều 57 Nghị_định này ; e ) Chánh Thanh_tra Sở Nông_nghiệp và_Phát_triển_nông_thôn , Chi_cục trưởng Chi_cục về bảo_vệ thực_vật , thú_y , thủy_sản thuộc Sở Nông_nghiệp và_Phát_triển_nông_thôn ; Trưởng_đoàn thanh_tra chuyên_ngành Tổng_cục Thủy_sản , Cục Thú_y , Cục Bảo_vệ thực_vật ; Trưởng_đoàn thanh_tra chuyên_ngành Sở Nông_nghiệp và_Phát_triển_nông_thôn xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại Điều 14 , 15 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 Nghị_định này trong trồng_trọt , chăn_nuôi , nuôi_trồng thủy_sản , thú_y , bảo_vệ thực_vật , bảo_quản , chế_biến nông_sản , lâm_sản , hải_sản và thực_phẩm ; g ) Trưởng_đoàn thanh_tra chuyên_ngành về hóa_chất của Bộ Công_Thương xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại Điều 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 ; khoản 1 và điểm a , b , c khoản 2 Điều 19 ; Điều 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 31 ; khoản 1 và điểm a , b , c khoản 2 Điều 32 ; Điều 33 , 34 , 35 và 36 Nghị_định này ; h ) Trưởng_đoàn thanh_tra chuyên_ngành Bộ Y_tế xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại Điều 14 , 15 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 Nghị_định này trong lĩnh_vực y_tế , hóa_chất sử_dụng trong chất diệt khuẩn , diệt côn_trùng trong gia_dụng và y_tế , dược_phẩm và phụ_gia thực_phẩm ; các hành_vi vi_phạm hành_chính quy_định tại Điều 37 ; khoản 1 , 2 , điểm a , b , c , d , đ , e_khoản 3 , điểm a , b , c , d , đ , e_khoản 4 , khoản 5 Điều 38 ; Điều 39 , 40 , 41 , 42 , 43 , 44 , 45 , 46 , 47 và 48 Nghị_định này ; i ) Trưởng_đoàn thanh_tra chuyên_ngành của Bộ_Nông_nghiệp và_Phát_triển_nông_thôn xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại Điều 14 , 15 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 Nghị_định này trong trồng_trọt , chăn_nuôi , nuôi_trồng thủy_sản , thú_y , bảo_vệ thực_vật , bảo_quản , chế_biến nông_sản , lâm_sản , hải_sản và thực_phẩm ; k ) Trưởng_đoàn thanh_tra chuyên_ngành về vật_liệu nổ công_nghiệp của Bộ Công_Thương xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại Điều 49 , 50 ; khoản 1 , 2 và 3 Điều 51 ; khoản 1 , 2 và 3 Điều 52 ; khoản 1 , 2 , 3 , 4 và 5 Điều 53 ; khoản 1 , 2 , 3 , 4 , 5 và 6 Điều 54 ; khoản 1 , 2 , 3 và 4 Điều 55 ; khoản 1 , 2 , 3 , 4 và 5 Điều 56 và Điều 57 Nghị_định này ; l ) Chánh Thanh_tra Bộ_Công_Thương xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại Mục 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 và 10 Chương II Nghị_định này ; m ) Chánh Thanh_tra Bộ Y_tế , Cục trưởng Cục Quản_lý môi_trường y_tế xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại Điều 14 , 15 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 Nghị_định này trong lĩnh_vực y_tế , hóa_chất sử_dụng trong chất diệt khuẩn , diệt côn_trùng trong gia_dụng và y_tế , dược_phẩm và phụ_gia thực_phẩm ; các hành_vi vi_phạm hành_chính quy_định tại Mục 9 Chương II Nghị_định này ; n ) Cục trưởng Cục Hóa_chất xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại Mục 1 , 2 , 3 , 4 , 5 , 6 , 7 và 8 Chương II Nghị_định này ; các hành_vi vi_phạm hành_chính về kinh_doanh , xuất_khẩu , nhập_khẩu tiền chất thuốc_nổ , xuất_khẩu , nhập_khẩu vật_liệu nổ công_nghiệp tại điểm b , d , đ khoản 1 , khoản 2 , điểm b , c , d khoản 3 Điều 49 ; Điều 50 ; điểm a_khoản 2 , khoản 3 Điều 51 ; khoản 1 , 2 , 3 , 5 và 6 Điều 53 ; Điều 55 Nghị_định này ; o ) Cục trưởng Cục Kỹ_thuật an_toàn và_Môi_trường công_nghiệp xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại Điều 5 ; Điều 8 ; Điều 11 ; Điều 14 , 15 ; khoản 1 Điều 30 Nghị_định này ; các hành_vi vi_phạm hành_chính quy_định tại Mục 10 Chương II Nghị_định này ; p ) Chánh Thanh_tra Bộ_Nông_nghiệp và_Phát_triển_nông_thôn , Tổng_cục trưởng Tổng_cục Thủy_sản , Cục trưởng Cục Thú_y , Cục trưởng Cục Bảo_vệ thực_vật xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại Điều 14 , 15 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 Nghị_định này trong trồng_trọt , chăn_nuôi , nuôi_trồng thủy_sản , thú_y , bảo_vệ thực_vật , bảo_quản , chế_biến nông_sản , lâm_sản , hải_sản và thực_phẩm ; q ) Chánh thanh_tra Bộ Giao_thông vận_tải , Bộ_Khoa_học_và Công_nghệ , Bộ_Tài_nguyên_và_Môi_trường , Bộ Thông_tin và Truyền_thông ; Tổng_cục trưởng Tổng_cục Đường_bộ Việt_Nam , Tổng_cục trưởng Tổng_cục Tiêu_chuẩn đo_lường chất_lượng , Tổng_cục trưởng Tổng_cục Môi_trường ; Cục trưởng Cục Đường_sắt Việt_Nam , Cục trưởng Cục Đường_thủy nội_địa Việt_Nam , Cục trưởng Cục Hàng_hải Việt_Nam , Cục trưởng Cục Hàng_không Việt_Nam , Cục trưởng Cục An_toàn bức_xạ và hạt_nhân , Cục trưởng Cục Viễn_thông , Cục trưởng Cục Phát_thanh , Truyền_hình và Thông_tin điện_tử , Cục trưởng Cục Báo_chí , Cục trưởng Cục Xuất_bản , in và Phát_hành , Cục trưởng Cục Quản_lý dược , Cục trưởng Cục Y_tế dự_phòng và các chức_danh tương_đương được Chính_phủ giao thực_hiện chức_năng thanh_tra chuyên_ngành xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại Chương II Nghị_định này theo quy_định tại khoản 4 Điều 59 và chức_năng , nhiệm_vụ , quyền_hạn được giao .']]
a

[['Thẩm_quyền xử_phạt vi_phạm hành_chính của Thanh_tra : a ) Thanh_tra_viên , người được giao thực_hiện nhiệm_vụ thanh_tra chuyên_ngành về hóa_chất đang thi_hành công_vụ xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại khoản 1 và 2 Điều 21 và Điều 36 Nghị_định này ; b ) Chánh Thanh_tra Sở Công_Thương , Trưởng_đoàn thanh_tra chuyên_ngành Sở Công_Thương xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại Điều 5 , 6 , 7 , 8 , 9 , 10 ; khoản 1 , điểm a , b , c , d khoản 2 , khoản 3 , 4 , 5 , 6 và 7 Điều 11 ; Điều 12 , 13 , 14 , 15 , 16 , 17 , 18 ; khoản 1 và điểm a , b khoản 2 Điều 19 ; Điều 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 31 ; khoản 1 và điểm a , b khoản 2 Điều 32 ; Điều 33 ; khoản 1 , 2 , 3 Điều 34 ; Điều 35 , 36 , 49 , 50 ; khoản 1 , 2 và 3 Điều 51 ; khoản 1 , 2 và 3 Điều 52 ; khoản 1 , 2 , 3 , 4 Điều 53 ; khoản 1 , 2 , 3 , 4 , 5 Điều 54 ; khoản 1 , 2 , 3 và 4 Điều 55 ; khoản 1 , 2 , 3 , 4 Điều 56 và Điều 57 Nghị_định này ; c ) Chánh Thanh_tra S

In [29]:
def preserve_structure_split_text(data):
    result = []
    for sublist in data:
        temp_result = []
        for text in sublist:
            # Check if the text contains patterns like 'a )', 'b )', etc.
            if re.search(r'(?<![a-zđ]) [a-zđ]\ \)', text, re.IGNORECASE):  #handle case like ' ( ban ngay )'
                # Splitting based on the patterns
                split_parts = re.split(r'((?<![a-zđ]) [a-zđ]\ \))', text, flags=re.IGNORECASE)
                for i in range(0, len(split_parts) - 1, 2):
                    temp_result.append(split_parts[i].strip() + split_parts[i + 1])
                if len(split_parts) % 2 != 0:
                    temp_result.append(split_parts[-1].strip())
            else:
                # If no patterns found, keep the text as is
                temp_result.append(text)
        result.append(temp_result)
    return result


conditional_split_result_fixed = preserve_structure_split_text(a)
conditional_split_result_fixed

[['Thẩm_quyền xử_phạt vi_phạm hành_chính của Thanh_tra : a )',
  'Thanh_tra_viên , người được giao thực_hiện nhiệm_vụ thanh_tra chuyên_ngành về hóa_chất đang thi_hành công_vụ xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại khoản 1 và 2 Điều 21 và Điều 36 Nghị_định này ; b )',
  'Chánh Thanh_tra Sở Công_Thương , Trưởng_đoàn thanh_tra chuyên_ngành Sở Công_Thương xử_phạt đối_với các hành_vi vi_phạm hành_chính quy_định tại Điều 5 , 6 , 7 , 8 , 9 , 10 ; khoản 1 , điểm a , b , c , d khoản 2 , khoản 3 , 4 , 5 , 6 và 7 Điều 11 ; Điều 12 , 13 , 14 , 15 , 16 , 17 , 18 ; khoản 1 và điểm a , b khoản 2 Điều 19 ; Điều 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 31 ; khoản 1 và điểm a , b khoản 2 Điều 32 ; Điều 33 ; khoản 1 , 2 , 3 Điều 34 ; Điều 35 , 36 , 49 , 50 ; khoản 1 , 2 và 3 Điều 51 ; khoản 1 , 2 và 3 Điều 52 ; khoản 1 , 2 , 3 , 4 Điều 53 ; khoản 1 , 2 , 3 , 4 , 5 Điều 54 ; khoản 1 , 2 , 3 và 4 Điều 55 ; khoản 1 , 2 , 3 , 4 Điều 56 và Điều 57 Nghị_định này ; c )',
  'Ch

In [30]:
sentence_tokenized_filter[8]

['Phạm_vi tuần_tra : a ) Báo_động lũ ở cấp I , bố_trí người tuần_tra như sau : - Lượt đi : 01 người ( ban_ngày ) , 02 người ( ban_đêm ) kiểm_tra mặt đê , mái đê phía sông , khu_vực hành_lang bảo_vệ đê phía sông ; - Lượt về : 01 người ( ban_ngày ) , 02 người ( ban_đêm ) kiểm_tra mái đê phía đồng , khu_vực hành_lang bảo_vệ đê phía đồng , mặt ruộng , hồ ao gần chân đê phía đồng ; b ) Báo_động lũ ở cấp II , bố_trí người tuần_tra như sau : - Lượt đi : 01 người kiểm_tra mặt đê , mái đê phía sông , khu_vực hành_lang bảo_vệ đê phía sông ; 01 người ( ban_ngày ) , 02 người ( ban_đêm ) kiểm_tra mái đê phía đồng , khu_vực hành_lang bảo_vệ đê phía đồng , mặt ruộng , hồ ao gần chân đê phía đồng ; - Lượt về : 01 người ( ban_ngày ) , 02 người ( ban_đêm ) kiểm_tra mặt đê , mái đê phía sông , khu_vực hành_lang bảo_vệ đê phía sông ; 01 người kiểm_tra mái đê phía đồng , khu_vực hành_lang bảo_vệ đê phía đồng , mặt ruộng , hồ ao gần chân đê phía đồng ; c ) Báo_động lũ ở cấp II và có tin_bão khẩn_cấp đổ_bộ v

In [31]:
conditional_split_result_fixed2 = preserve_structure_split_text(sentence_tokenized_filter[7:9])
conditional_split_result_fixed2

[['Báo_động lũ ở cấp I : Bố_trí ngày 02 người , đêm 04 người .',
  'Ban_ngày ít_nhất sau 04 giờ có 01 kíp đi tuần , mỗi kíp có 01 người .',
  'Ban_đêm ít_nhất sau 04 giờ có 01 kíp đi tuần , mỗi kíp 02 người .',
  'Báo_động lũ ở cấp II : a )',
  'Bố_trí ngày 04 người , đêm 06 người .',
  'Ban_ngày ít_nhất sau 02 giờ có 01 kíp đi tuần , mỗi kíp 02 người .',
  'Ban_đêm ít_nhất sau 02 giờ có 01 kíp đi tuần , mỗi kíp 03 người ; b )',
  'Trường_hợp có tin_bão khẩn_cấp đổ_bộ vào khu_vực : bố_trí ngày 06 người , đêm 12 người , chia thành các kíp , mỗi kíp 03 người ; tùy theo diễn_biến của bão , lũ và đặc_điểm của tuyến đê , Ban chỉ_huy phòng , chống_lụt , bão cấp xã quyết_định việc tăng_cường số lần kiểm_tra so với quy_định tại điểm a_khoản 2 Điều này .',
  'Báo_động lũ ở cấp III trở lên : a )',
  'Bố_trí ngày 06 người , đêm 12 người , chia thành các kíp , mỗi kíp 03 người , không phân_biệt ngày_đêm các kíp phải liên_tục thay phiên nhau kiểm_tra ; b )',
  'Đối_với các vị_trí xung_yếu của đê , 

In [32]:
# Split a ), b ) into multiple sentence, but there still exists elements like 'b )', 'c )'
sentence_truncate_filter = preserve_structure_split_text(sentence_tokenized_filter)

In [33]:
max_sentence_length_after_truncate = max(len(sentence.split()) for document in sentence_truncate_filter for sentence in document)
print(f'Max sentence length: {max_sentence_length_after_truncate}')

Max sentence length: 1141


In [34]:
sentence_lengths_after_truncate = [len(sentence.split()) for document in sentence_truncate_filter for sentence in document]

# Find the top 10 maximum lengths
top_10_lengths_after_truncate = sorted(sentence_lengths_after_truncate, reverse=True)[:10]

print("Top 10 maximum lengths:", top_10_lengths_after_truncate)

Top 10 maximum lengths: [1141, 983, 831, 695, 636, 602, 597, 596, 589, 588]


In [35]:
sentence_length_pairs_after_truncate = [(sentence, len(sentence.split())) for document in sentence_truncate_filter for sentence in document]
top_10_sentences_after_truncate = sorted(sentence_length_pairs_after_truncate, key=lambda x: x[1], reverse=True)[:10]
print("Top 10 sentences by length:")
for sentence, length in top_10_sentences_after_truncate:
    print(f"Length: {length}, Sentence: {sentence}")

Top 10 sentences by length:
Length: 1141, Sentence: Cầu_vai , cấp hàm_a ) Cầu_vai , cấp hàm đối_với cán_bộ , thanh_tra_viên giữ chức_vụ lãnh_đạo quản_lý - Tổng_Thanh_tra Chính_phủ : cầu vai có chiều dài 13 cm , phần đầu nhọn rộng 4 cm , phần đuôi rộng 5 cm , nền cấp_hiệu bằng vải nỉ màu đỏ , cốt cấp_hiệu bằng nhựa nguyên_chất màu trắng ; sao và viền cấp hiệu bằng đồng tấm , viền kim_loại mạ màu trắng bóng ; cúc cấp_hiệu hình_tròn có ngôi_sao ở giữa 2 cành_tùng , gắn 4 sao 23 mm thành một hàng dọc cầu vai ; - Phó Tổng_Thanh_tra Chính_phủ : cầu vai có chiều dài 13 cm , phần đầu nhọn rộng 4 cm , phần đuôi rộng 5 cm , nền cấp_hiệu bằng vải nỉ màu đỏ , cốt cấp_hiệu bằng nhựa nguyên_chất màu trắng ; sao và viền cấp hiệu bằng đồng tấm , viền kim_loại mạ màu trắng bóng ; cúc cấp_hiệu hình_tròn có ngôi_sao ở giữa 2 cành_tùng ; nhiệm_kỳ 1 có cấp hàm gắn 2 sao , từ nhiệm_kỳ 2 trở lên cấp hàm có gắn 3 sao 23 mm thành một hàng dọc cầu vai ; - Vụ trưởng và tương_đương thuộc Thanh_tra Chính_phủ ; Chá

In [36]:
b= [['Kiến_thức - Mô_tả được vòng tuần hoàn_sinh_thái , nguyên_nhân và tác_động của ô_nhiễm môi_trường và các biện_pháp giảm_thiểu ô_nhiễm môi_trường ; - Trình_bày được các loại , cấu_tạo và điều_kiện sống của vi_sinh_vật cũng như vai_trò của chúng trong xử_lý nước_thải ; - Trình_bày được các bước chuẩn_bị thực_hiện công_việc vận_hành , bảo_trì , bảo_dưỡng mạng_lưới thoát nước và nhà_máy xử_lý nước_thải ; - Phân_tích được các biện_pháp đảm_bảo an_toàn trong quá_trình vận_hành , bảo_trì , bảo_dưỡng mạng_lưới thoát nước và nhà_máy xử_lý nước_thải ; - Phân_tích được bản_vẽ hiện_trạng mạng_lưới thoát nước , sơ_đồ mạch điện và các tài_liệu kỹ_thuật khác ; - Trình_bày các biện_pháp tiết_kiệm năng_lượng , vật_tư trong vận_hành , bảo_trì , bảo_dưỡng mạng_lưới thoát nước và nhà_máy xử_lý nước_thải ; - Đánh_giá được thành_phần , tính_chất vật_lý , hóa_học , sinh_học của các loại nước_thải ; - Trình_bày được cấu_tạo , đặc_tính , ưu và nhược_điểm của các loại hệ_thống thoát nước ; - Mô_tả được đặc_tính và phạm_vi sử_dụng của các loại vật_liệu ống , cống thoát nước ; - Trình_bày được chức_năng , nhiệm_vụ và các hư_hỏng thông_thường của các công_trình trên mạng_lưới thoát nước và trong nhà_máy xử_lý nước_thải ; - Trình_bày được nguyên_nhân , hậu_quả và các biện_pháp khắc_phục của các hư_hại thường_xuyên xảy ra trên mạng_lưới thoát nước và nhà_máy xử_lý nước_thải ; - Mô_tả được các nguy_cơ gây bệnh và mối nguy_hiểm có_thể xảy ra khi vận_hành , bảo_trì , bảo_dưỡng mạng_lưới thoát nước , nhà_máy xử_lý nước_thải ; - Mô_tả được các phương_pháp đo , điều_khiển , điều_chỉnh và cấu_tạo , chức_năng của các thiết_bị tương_ứng ; - Trình_bày được cách tính_toán độ dốc của cống thoát nước và cách tính khoảng_cách , độ sâu của hố_ga ; - Trình_bày được các biện_pháp vệ_sinh , bảo_trì , bảo_dưỡng đường cống , nắp cống chịu lực , hố_ga , điểm đấu_nối , trạm bơm .. ; - Trình_bày được sơ_đồ tổng_quan của một nhà_máy xử_lý nước_thải , các bậc xử_lý trong nhà_máy xử_lý nước_thải sinh_hoạt và nhà_máy xử_lý nước_thải công_nghiệp đặc_thù ; - Phân_tích được các quá_trình phân_hủy của các chất ô_nhiễm trong từng công_trình và phương_pháp loại_bỏ chúng ; - Trình_bày được cấu_tạo , chức_năng và nguyên_lý hoạt_động các công_trình và thiết_bị xử_lý nước_thải bằng phương_pháp cơ_học , hóa_học , hóa_lý và sinh_học , công_trình khử_trùng , khử mùi ; - Trình_bày được quy_trình vận_hành , bảo_trì , bảo_dưỡng các công_trình xử_lý nước_thải , bùn_thải , rác_thải và khí_thải trong nhà_máy ; - Liệt_kê được các nguồn phát_sinh , tính_chất và ảnh_hưởng của bùn_thải , khí_thải và rác_thải trong hệ_thống xử_lý nước_thải ; - Trình_bày được quyền_lợi , nghĩa_vụ cơ_bản của người lao_động và người sử_dụng lao_động , các tiêu_chuẩn , quy_chuẩn kỹ_thuật quốc_gia áp_dụng cho chuyên_ngành thoát và xử_lý nước_thải , bảo_vệ môi_trường cũng như quy_chuẩn , tiêu_chuẩn về an_toàn ; - Trình_bày được tên , đặc_tính , quy_trình vận_hành , điều_kiện sử_dụng , bảo_quản của thiết_bị , dụng_cụ , hóa_chất trong phòng_thí_nghiệm ; - Trình_bày được các phương_pháp lấy , vận_chuyển , lưu_trữ và bảo_quản mẫu nước_thải và bùn ; - Trình_bày phương_pháp phân_tích chỉ_tiêu tại hiện_trường , trong nhà_máy và phòng_thí_nghiệm ứng với từng thông_số cần xác_định ; - Trình_bày được cơ_cấu chung , chức_năng , nhiệm_vụ bảo_vệ môi_trường của các doanh_nghiệp hoạt_động trong lĩnh_vực thoát nước và xử_lý nước_thải ; - Trình_bày được những kiến_thức cơ_bản về chính_trị , văn_hóa , xã_hội , pháp_luật , quốc_phòng an_ninh , giáo_dục thể_chất theo quy_định .']]
b

[['Kiến_thức - Mô_tả được vòng tuần hoàn_sinh_thái , nguyên_nhân và tác_động của ô_nhiễm môi_trường và các biện_pháp giảm_thiểu ô_nhiễm môi_trường ; - Trình_bày được các loại , cấu_tạo và điều_kiện sống của vi_sinh_vật cũng như vai_trò của chúng trong xử_lý nước_thải ; - Trình_bày được các bước chuẩn_bị thực_hiện công_việc vận_hành , bảo_trì , bảo_dưỡng mạng_lưới thoát nước và nhà_máy xử_lý nước_thải ; - Phân_tích được các biện_pháp đảm_bảo an_toàn trong quá_trình vận_hành , bảo_trì , bảo_dưỡng mạng_lưới thoát nước và nhà_máy xử_lý nước_thải ; - Phân_tích được bản_vẽ hiện_trạng mạng_lưới thoát nước , sơ_đồ mạch điện và các tài_liệu kỹ_thuật khác ; - Trình_bày các biện_pháp tiết_kiệm năng_lượng , vật_tư trong vận_hành , bảo_trì , bảo_dưỡng mạng_lưới thoát nước và nhà_máy xử_lý nước_thải ; - Đánh_giá được thành_phần , tính_chất vật_lý , hóa_học , sinh_học của các loại nước_thải ; - Trình_bày được cấu_tạo , đặc_tính , ưu và nhược_điểm của các loại hệ_thống thoát nước ; - Mô_tả được đặc_tí

In [37]:
def split_text_multiple_dashes(data):
    result = []
    for sublist in data:
        temp_result = []
        for text in sublist:
            # Check if the text contains any dashes ('-')
            if '-' in text:
                # Splitting at each occurrence of '-'
                split_parts = text.split('-')
                for i, part in enumerate(split_parts):
                    if i == 0:
                        temp_result.append(part.strip())  # The text before the first dash
                    else:
                        temp_result.append('- ' + part.strip())  # Prefix the text after each dash
            else:
                # If no dash found, keep the text as is
                temp_result.append(text)
        result.append(temp_result)
    return result


split_result_with_dash = split_text_multiple_dashes(b)
split_result_with_dash

[['Kiến_thức',
  '- Mô_tả được vòng tuần hoàn_sinh_thái , nguyên_nhân và tác_động của ô_nhiễm môi_trường và các biện_pháp giảm_thiểu ô_nhiễm môi_trường ;',
  '- Trình_bày được các loại , cấu_tạo và điều_kiện sống của vi_sinh_vật cũng như vai_trò của chúng trong xử_lý nước_thải ;',
  '- Trình_bày được các bước chuẩn_bị thực_hiện công_việc vận_hành , bảo_trì , bảo_dưỡng mạng_lưới thoát nước và nhà_máy xử_lý nước_thải ;',
  '- Phân_tích được các biện_pháp đảm_bảo an_toàn trong quá_trình vận_hành , bảo_trì , bảo_dưỡng mạng_lưới thoát nước và nhà_máy xử_lý nước_thải ;',
  '- Phân_tích được bản_vẽ hiện_trạng mạng_lưới thoát nước , sơ_đồ mạch điện và các tài_liệu kỹ_thuật khác ;',
  '- Trình_bày các biện_pháp tiết_kiệm năng_lượng , vật_tư trong vận_hành , bảo_trì , bảo_dưỡng mạng_lưới thoát nước và nhà_máy xử_lý nước_thải ;',
  '- Đánh_giá được thành_phần , tính_chất vật_lý , hóa_học , sinh_học của các loại nước_thải ;',
  '- Trình_bày được cấu_tạo , đặc_tính , ưu và nhược_điểm của các loại h

In [38]:
# Remove multiple dash to seperate sentence
sentence_truncate_remove_dash = split_text_multiple_dashes(sentence_truncate_filter)

In [39]:
max_sentence_length_after_remove_dash = max(len(sentence.split()) for document in sentence_truncate_remove_dash for sentence in document)
print(f'Max sentence length: {max_sentence_length_after_remove_dash}')

Max sentence length: 558


In [40]:
sentence_lengths_after_remove_dash = [len(sentence.split()) for document in sentence_truncate_remove_dash for sentence in document]

# Find the top 10 maximum lengths
top_10_lengths_after_remove_dash = sorted(sentence_lengths_after_remove_dash, reverse=True)[:10]

print("Top 10 maximum lengths:", top_10_lengths_after_remove_dash)

Top 10 maximum lengths: [558, 506, 504, 502, 496, 492, 491, 478, 477, 476]


In [41]:
sentence_length_pairs_after_remove_dash = [(sentence, len(sentence.split())) for document in sentence_truncate_remove_dash for sentence in document]
top_10_sentences_after_remove_dash = sorted(sentence_length_pairs_after_remove_dash, key=lambda x: x[1], reverse=True)[:10]
print("Top 10 sentences by length:")
for sentence, length in top_10_sentences_after_remove_dash:
    print(f"Length: {length}, Sentence: {sentence}")

Top 10 sentences by length:
Length: 558, Sentence: Những vấn_đề chưa hoàn_thiện được ( nêu rõ lý_do ) : … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … XÁC_NHẬN CỦA ĐƠN_VỊ CHỦ_TRÌ NHIỆM_VỤ ( Họ , tên , chữ_ký và đóng_dấu ) CHỦ_NHIỆM NHIỆM_VỤ ( Họ , tên và chữ_ký ) Ý_kiến của Ủy_viên phản_biện 1 : … … … … … … … … … … … … … … … … … … … … ... … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … … ( Họ

In [42]:
len(sentence_truncate_remove_dash)

261597

In [43]:
sentence_truncate_remove_dash[0:10]

[['Thông_tư này hướng_dẫn tuần_tra , canh_gác bảo_vệ đê_Điều trong mùa lũ đối_với các tuyến đê sông được phân_loại , phân_cấp theo quy_định tại Điều 4 của Luật Đê_Điều .'],
 ['Hàng năm trước mùa mưa , lũ , Ủy_ban_nhân_dân cấp xã nơi có đê phải tổ_chức lực_lượng lao_động tại địa_phương để tuần_tra , canh_gác đê và thường_trực trên các điếm canh đê hoặc nhà dân khu_vực gần đê ( đối_với những khu_vực chưa có điếm canh đê ) , khi có báo_động lũ từ cấp I trở lên đối_với tuyến sông có đê ( sau đây gọi tắt là lực_lượng tuần_tra , canh_gác đê ) .',
  'Lực_lượng tuần_tra , canh_gác đê được tổ_chức thành các đội , do Ủy_ban_nhân_dân cấp xã ra quyết_định thành_lập ; từ 01 đến 02 kilômét đê thành_lập 01 đội ; mỗi đội có từ 12 đến 18 người , trong đó có 01 đội_trưởng và 01 hoặc 02 đội phó .',
  'Danh_sách thành_viên đội tuần_tra , canh_gác đê được niêm_yết tại điếm canh đê thuộc địa_bàn được phân_công .',
  'Khi lũ , bão có diễn_biến phức_tạp , kéo_dài ngày , Ủy_ban_nhân_dân cấp xã có_thể quyết_địn

In [44]:
def remove_redundant_elements(data):
    result = []
    for sublist in data:
        filtered_sublist = [text for text in sublist if len(text) >= 10]
        result.append(filtered_sublist)
    return result

In [45]:
final_version = remove_redundant_elements(sentence_truncate_remove_dash)
final_version[0:10]

[['Thông_tư này hướng_dẫn tuần_tra , canh_gác bảo_vệ đê_Điều trong mùa lũ đối_với các tuyến đê sông được phân_loại , phân_cấp theo quy_định tại Điều 4 của Luật Đê_Điều .'],
 ['Hàng năm trước mùa mưa , lũ , Ủy_ban_nhân_dân cấp xã nơi có đê phải tổ_chức lực_lượng lao_động tại địa_phương để tuần_tra , canh_gác đê và thường_trực trên các điếm canh đê hoặc nhà dân khu_vực gần đê ( đối_với những khu_vực chưa có điếm canh đê ) , khi có báo_động lũ từ cấp I trở lên đối_với tuyến sông có đê ( sau đây gọi tắt là lực_lượng tuần_tra , canh_gác đê ) .',
  'Lực_lượng tuần_tra , canh_gác đê được tổ_chức thành các đội , do Ủy_ban_nhân_dân cấp xã ra quyết_định thành_lập ; từ 01 đến 02 kilômét đê thành_lập 01 đội ; mỗi đội có từ 12 đến 18 người , trong đó có 01 đội_trưởng và 01 hoặc 02 đội phó .',
  'Danh_sách thành_viên đội tuần_tra , canh_gác đê được niêm_yết tại điếm canh đê thuộc địa_bàn được phân_công .',
  'Khi lũ , bão có diễn_biến phức_tạp , kéo_dài ngày , Ủy_ban_nhân_dân cấp xã có_thể quyết_địn

In [46]:
max_num_sentences = [len(document) for document in final_version]
top_10_num_sentences = sorted(max_num_sentences, reverse=True)[:10]

print("Top 10 maximum number of sentences:", top_10_num_sentences)

Top 10 maximum number of sentences: [2616, 2223, 1033, 829, 703, 536, 434, 395, 359, 356]


In [47]:
num_sentences_pairs = [(index, document, len(document)) for index, document in enumerate(final_version)]
top_10_number_of_sentences = sorted(num_sentences_pairs, key=lambda x: x[2], reverse=True)[:10]
print(f"Index: {top_10_number_of_sentences[0][0]}, Number of sentences: {top_10_number_of_sentences[0][-1]}, Document: {top_10_number_of_sentences[0][1]}")

Index: 25326, Number of sentences: 2616, Document: ['Tài_khoản 111', '- Tiền_mặt Tài_khoản này dùng để phản_ánh tình_hình thu , chi , tồn tiền_mặt tại quỹ của BQLDAĐT , bao_gồm : Tiền Việt_Nam , ngoại_tệ .', 'Nguyên_tắc hạch_toán', '- Chỉ phản_ánh vào Tài_khoản 111 “ Tiền_mặt ” số tiền_mặt , ngoại_tệ thực_tế nhập , xuất_quỹ tiền_mặt .', 'Đối_với những khoản tiền thu được chuyển nộp ngay vào Ngân_hàng , Kho_bạc Nhà_nước ( không qua quỹ tiền_mặt của đơn_vị ) thì không ghi vào bên Nợ Tài_khoản 111 “ Tiền_mặt ” mà ghi vào bên Nợ Tài_khoản 113 “ Tiền đang chuyển ” .', '- Các khoản tiền_mặt do doanh_nghiệp khác và cá_nhân ký_cược , ký quỹ tại đơn_vị được quản_lý và hạch_toán như các loại tài_sản bằng tiền của đơn_vị .', '- Khi tiến_hành nhập , xuất_quỹ tiền_mặt phải có phiếu thu , phiếu chi và có đủ chữ_ký của người nhận , người giao , người cho_phép nhập , xuất_quỹ theo quy_định của chế_độ chứng_từ kế_toán .', 'Một_số trường_hợp phải có lệnh nhập_quỹ , xuất_quỹ đính kèm .', '- Kế_toán quỹ t

In [48]:
with open ('sentence_tokenized_final.pkl', 'wb') as f:
    pickle.dump(final_version, f)